# 기본 CF 알고리즘

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


def get_dataset_1() -> "pd.DataFrame":
  """users, movies, ratings dataframe을 반환하는 함수

  Returns:
      pd.DataFrame: users, movies, ratings dataframe
  """
  u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
  i_cols = ['movie_id', 'title', 'release_date', 'video release date', 'IMDB URL', 'unknown',
            'Action', 'Adventure', 'Animation', 'children\s', 'Comedy', 'Crime', 'Documentary', 'Drama',
            'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
            'Western']
  r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

  users = pd.read_csv('../data2/u.user', sep='|', names=u_cols, encoding='latin-1')
    
  movies = pd.read_csv('../data2/u.item', sep='|', names=i_cols, encoding='latin-1')
  movies = movies[['movie_id', 'title']]
  
  ratings = pd.read_csv('../data2/u.data', sep='\t', names=r_cols, encoding='latin-1')
  ratings.drop('timestamp', axis=1, inplace=True)
  
  return users, movies, ratings

users, movies, ratings = get_dataset_1()

X = ratings.copy()
y = ratings['user_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

rating_matrix = X_train.pivot(index="user_id", columns='movie_id', values='rating')

def RMSE(y_true:"pd.Series"or "np.array", y_pred:"pd.Series") -> float:
  """_summary_

  Args:
      y_true (pd.Series&quot;or&quot;np.array): y의 정답
      y_pred (pd.Series&quot;or&quot;np.array): y의 예측치

  Returns:
      float: RMSE
  """
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
def score(model) -> float:
  """모델별 RMSE를 계산하는 함수

  Args:
      model (_type_): rating을 예측하는 모델

  Returns:
      float: RMSE
  """
  id_pairs = zip(X_test['user_id'], X_test['movie_id'])
  
  y_pred = np.array([model(user_id, movie_id) for (user_id, movie_id) in id_pairs])
  y_true = np.array(X_test['rating'])

  return RMSE(y_true, y_pred)



In [5]:
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = rating_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [32]:
# 이웃을 전체 사용자로 설정
def CF_simple(user_id:str, movie_id:str) -> float:
  """이웃을 전체 사용자로 설정하고, 
  주어진 영화에 대해서 평가한 각 사용자에 대해서, 평점을 유사도로 가중평균한 예측치를 구함.
  즉, 해당 user id가 movie id를 어떻게 평가할 것인지를 유사도로 평점을 가중평균해 예측하는 함수 

  Args:
      user_id (str): 사용자 id
      movie_id (str): movie id

  Returns:
      float: user id와 movie id를 평가한 사용자에 대한, 유사도로 평점을 가중평균한 예측치
  """
  # 해당 movie id에 대해서 평가한 값이 있는지 확인
  if movie_id in rating_matrix.columns:
    movie_ratings = rating_matrix[movie_id].copy()
    # movie_id에 대해서 평가하지 않은 user 
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    
    sim_scores = user_similarity[user_id].copy()
    sim_scores = sim_scores.dropna()
    # 평가하지 않은 유저는 뺴준다.
    sim_scores = sim_scores.drop(none_rating_idx, axis=0)
    
    # 주어진 영화에 대해서 평가한 각 사용자에 대해서 평점을 유사도로 가중평균한 예측치를 구함
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  # 없으면 3.0으로 예측
  else:
    mean_rating = 3.0
    
  return mean_rating

In [33]:
score(CF_simple)

1.0166301998153813

연습문제)   
위의 코드를 수정해서 코사인 유사도 대신에 피어슨 상관계수를 사용하는 코드를 작성하고 RMSE를 계산하세요.

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = rating_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

user_corr = matrix_dummy.T.corr()
user_corr = pd.DataFrame(user_corr, index=rating_matrix.index, columns=rating_matrix.index)

def CF_simple_using(simil:str):
  """similarity를 어떻게 계산할지 여부에 따라 해당 model을 return하는 함수

  Args:
      simil (str): similarity계산 방식 ( cosine or corr )

  Raises:
      Exception: simil값이 정확하지 않을 때

  Returns:
      _type_: CF_simple model
  """
  if simil == 'cosine':
    return CF_simple_cosine
  elif simil == 'corr':
    return CF_simple_corr
  else:
    raise Exception('simil값을 확인해주세요 (cosine or corr)')    

# 이웃을 전체 사용자로 설정
def CF_simple_cosine(user_id:str, movie_id:str) -> float:
  """이웃을 전체 사용자로 설정하고, 
  주어진 영화에 대해서 평가한 각 사용자에 대해서, 평점을 유사도로 가중평균한 예측치를 구함.
  즉, 해당 user id가 movie id를 어떻게 평가할 것인지를 유사도로 평점을 가중평균해 예측하는 함수 
  유사도 : cosine similarity
  Args:
      user_id (str): 사용자 id
      movie_id (str): 영화 id
  Returns:
      float: user id와 movie id를 평가한 사용자에 대한, 유사도로 평점을 가중평균한 예측치
  """
  # 해당 movie id에 대해서 평가한 값이 있는지 확인
  if movie_id in rating_matrix.columns:
    movie_ratings = rating_matrix[movie_id].copy()
    # movie_id에 대해서 평가하지 않은 user 
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    
    sim_scores = user_similarity[user_id].copy()
    sim_scores = sim_scores.dropna()
    # 평가하지 않은 유저는 뺴준다.
    sim_scores = sim_scores.drop(none_rating_idx, axis=0)
    
    # 주어진 영화에 대해서 평가한 각 사용자에 대해서 평점을 유사도로 가중평균한 예측치를 구함
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  # 없으면 3.0으로 예측
  else:
    mean_rating = 3.0
    
  return mean_rating

# 이웃을 전체 사용자로 설정
def CF_simple_corr(user_id:str, movie_id:str) -> float:
  """이웃을 전체 사용자로 설정하고, 
  주어진 영화에 대해서 평가한 각 사용자에 대해서, 평점을 유사도로 가중평균한 예측치를 구함.  
  즉, 해당 user id가 movie id를 어떻게 평가할 것인지를 유사도로 평점을 가중평균해 예측하는 함수.
  유사도: correlation
  
  Args:
      user_id (str): 사용자 id
      movie_id (str): 영화 id
  Returns:
      float: user id와 movie id를 평가한 사용자에 대한, 유사도로 평점을 가중평균한 예측치
  """
  # 해당 movie id에 대해서 평가한 값이 있는지 확인
  if movie_id in rating_matrix.columns:
    movie_ratings = rating_matrix[movie_id].copy()
    # movie_id에 대해서 평가하지 않은 user 
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    
    sim_scores = user_corr[user_id].copy()
    sim_scores = sim_scores.dropna()
    # 평가하지 않은 유저는 뺴준다.
    sim_scores = sim_scores.drop(none_rating_idx, axis=0)
    
    # 주어진 영화에 대해서 평가한 각 사용자에 대해서 평점을 유사도로 가중평균한 예측치를 구함
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  # 없으면 3.0으로 예측
  else:
    mean_rating = 3.0
    
  return mean_rating

In [49]:
print(f"smilarity: cosine , {score(CF_simple_using(simil='cosine')):.4f}")
print(f"smilarity: corr , {score(CF_simple_using(simil='corr')):.4f}")

smilarity: cosine , 1.0190
smilarity: corr , 1.3774


# 이웃을 고려한 CF